# Notebook to translate and process NTH1 QC data from raw files

assumes you. have the tar.gz files already locally

In [1]:
import os
from es_sfgtools.utils.archive_pull import load_site_metadata, load_vessel_metadata
from es_sfgtools.data_mgmt.datadiscovery import expand_tar_gz
from es_sfgtools.data_mgmt.data_handler import DataHandler
#from es_sfgtools.data_models.community_standards import SFGDSTFSeafloorAcousticData, SFGDTSFSite
#from es_sfgtools.data_models.observables import ShotDataFrame
import pandas as pd

In [2]:
# this must be set correctly for GO executables to translate novatel to rinex
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/Caskroom/miniconda/base/envs/seafloor_geodesy_mac/lib'
DYLD_LIBRARY_PATH = os.getenv("DYLD_LIBRARY_PATH")
print("DYLD_LIBRARY_PATH:", DYLD_LIBRARY_PATH)

DYLD_LIBRARY_PATH: /opt/homebrew/Caskroom/miniconda/base/envs/seafloor_geodesy_mac/lib


In [3]:
network_name = "cascadia-gorda"
site_name = "NTH1"
vessel = "1126"
year = "2025"
campaign_name = f"{year}_A_{vessel}"
data_dir = f"/Users/mikegottlieb/data/sfg"
qc_dir = f"{data_dir}/{network_name}/{site_name}/{campaign_name}/qc"
metadata_file = f"{data_dir}/{network_name}/{site_name}/{site_name}.json"
site_meta = load_site_metadata(station=site_name, network=network_name, local_path= metadata_file)

vessel_meta = load_vessel_metadata(vessel_code=vessel)

for campaign in site_meta.campaigns:
    if campaign.name == campaign_name:
        s = campaign.start
        e = campaign.end

dh = DataHandler(directory=data_dir)
dh.change_working_station(network=network_name, station=site_name, campaign=campaign_name, start_date=s, end_date=e)

qc_pipeline, config = dh.get_pipeline_sv3qc()



Loading vessel metadata from https://data.earthscope.org/archive/seafloor/metadata/vessels/1126.json
Loading vessel metadata from https://data.earthscope.org/archive/seafloor/metadata/vessels/1126.json
Built directory structure for cascadia-gorda NTH1 2025_A_1126
Creating TileDB arrays for NTH1
Consolidating existing TileDB arrays for NTH1
Changed working station to cascadia-gorda NTH1 2025_A_1126


In [ ]:
#if needed run this to extract tar.gz files
#qc_pins = expand_tar_gz(qc_dir)

In [ ]:

dh.discover_data_and_add_files(qc_dir)

In [ ]:
#read the pin files and use the onboard positions to populate the qcdata_pre array
config.qcpin_config.override = False
qc_pipeline.config = config
qc_pipeline.process_pin()

In [ ]:
config.qcpin_config.override = False
qc_pipeline.config = config

qc_pipeline.parse_rangea_logs_from_pin()

# qc_pins = dh.catalog.get_assets(network=network_name, station=site_name, campaign=campaign_name, type="qcpin")
# test_pin_path = qc_pins[3].local_path

# path = qcpin_to_novatelpin(test_pin_path, writedir=dh.inter_dir)
# path

In [ ]:
qc_pipeline.process_rangea_logs()

In [ ]:
qc_pipeline.get_rinex_files()

In [ ]:
qc_pipeline.process_rinex()

In [4]:
config.rinex_config.override = True
qc_pipeline.config = config
qc_pipeline.process_kin()

Looking for Kin Files to Process for cascadia-gorda NTH1 2025_A_1126
Found 9 Kin Files to Process: processing


Processing Kin Files:   0%|          | 0/9 [00:00<?, ?it/s]

Parsing KIN file: /Users/mikegottlieb/data/sfg/cascadia-gorda/NTH1/2025_A_1126/intermediate/kin_2025231_nth1.kin
Parsed 812 records from KIN file /Users/mikegottlieb/data/sfg/cascadia-gorda/NTH1/2025_A_1126/intermediate/kin_2025231_nth1.kin
Adding residuals from /Users/mikegottlieb/data/sfg/cascadia-gorda/NTH1/2025_A_1126/intermediate/res_2025231_nth1.res
Added WRMS residuals for 812 of 812 records
GNSS Parser: 812 shots from FILE /Users/mikegottlieb/data/sfg/cascadia-gorda/NTH1/2025_A_1126/intermediate/kin_2025231_nth1.kin
Parsing KIN file: /Users/mikegottlieb/data/sfg/cascadia-gorda/NTH1/2025_A_1126/intermediate/kin_2025227_nth1.kin
Parsed 701 records from KIN file /Users/mikegottlieb/data/sfg/cascadia-gorda/NTH1/2025_A_1126/intermediate/kin_2025227_nth1.kin
Adding residuals from /Users/mikegottlieb/data/sfg/cascadia-gorda/NTH1/2025_A_1126/intermediate/res_2025227_nth1.res
Added WRMS residuals for 701 of 701 records
GNSS Parser: 701 shots from FILE /Users/mikegottlieb/data/sfg/cascad

In [55]:
pd.options.display.float_format = '{:.3f}'.format  

qcdata_df = qc_pipeline.qcdata_pre.read_df(start=s)
qcdata_df_updated = qcdata_df.copy()
qcdata_df

,pingTime,transponderID,head0,pitch0,roll0,head1,pitch1,roll1,east0,north0,...,east_std1,north_std1,up_std1,returnTime,tt,dbv,xc,snr,tat,isUpdated
0,1755043235.000,IR5209,231.515,-1.093,2.938,234.359,1.085,0.824,-2741333.228,-3953327.696,...,7.791,9.097,23.000,1755043237.646,2.446,-26,88,-3.000,0.200,0
1,1755043355.000,IR5209,251.405,-1.823,0.761,243.732,-0.624,-1.449,-2741388.748,-3953286.568,...,7.773,9.126,23.000,1755043357.647,2.447,-25,86,30.000,0.200,0
2,1755043490.000,IR5209,270.209,1.621,1.289,260.865,2.179,-1.474,-2741441.316,-3953223.511,...,8.963,10.320,23.000,1755043492.620,2.420,-24,87,15.000,0.200,0
3,1755043610.000,IR5209,290.659,-1.561,0.711,297.256,0.943,2.361,-2741465.213,-3953157.069,...,8.953,10.266,23.000,1755043612.567,2.367,-25,91,31.000,0.200,0
4,1755043745.000,IR5209,330.694,-0.440,1.634,322.297,1.662,-2.183,-2741449.526,-3953079.576,...,8.887,10.225,23.000,1755043747.472,2.272,-21,79,22.000,0.200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,1755128526.000,IR5211,328.422,4.358,-0.358,332.253,-2.097,2.204,-2740984.088,-3952786.074,...,8.181,10.006,23.000,1755128529.080,2.640,-30,69,9.000,0.440,0
1338,1755128661.000,IR5211,335.374,6.832,-2.135,329.545,1.554,-0.995,-2740942.417,-3952727.958,...,7.785,9.009,23.000,1755128664.161,2.721,-34,65,10.000,0.440,0
1339,1755128781.000,IR5211,328.410,-3.194,5.020,339.906,-1.567,6.082,-2740903.400,-3952673.308,...,7.791,9.027,23.000,1755128784.244,2.804,-35,72,10.000,0.440,0
1340,1755128901.000,IR5211,331.807,-0.203,5.416,330.001,1.435,-0.040,-2740872.243,-3952626.320,...,7.809,9.062,20.700,1755128904.321,2.881,-36,70,7.000,0.440,0


In [56]:
qcdata_df['PingDateTime'] = pd.to_datetime(qcdata_df['pingTime'], unit='s')
qcdata_df['ReturnDateTime'] = pd.to_datetime(qcdata_df['returnTime'], unit='s')
qcdata_df['delta_east'] = qcdata_df['east1']-qcdata_df['east0']
qcdata_df['delta_north'] = qcdata_df['north1']-qcdata_df['north0']
qcdata_df['delta_up'] = qcdata_df['up1']-qcdata_df['up0']
qcdata_df[['pingTime','returnTime','transponderID','delta_east', 'delta_north', 'delta_up']]

,pingTime,returnTime,transponderID,delta_east,delta_north,delta_up
0,1755043235.000,1755043237.646,IR5209,-0.131,1.176,-0.367
1,1755043355.000,1755043357.647,IR5209,-2.154,0.165,0.709
2,1755043490.000,1755043492.620,IR5209,-0.636,0.685,1.300
3,1755043610.000,1755043612.567,IR5209,-0.574,2.225,1.552
4,1755043745.000,1755043747.472,IR5209,0.387,0.763,1.608
...,...,...,...,...,...,...
1337,1755128526.000,1755128529.080,IR5211,1.272,1.241,2.257
1338,1755128661.000,1755128664.161,IR5211,0.598,1.281,1.236
1339,1755128781.000,1755128784.244,IR5211,0.623,-0.955,1.768
1340,1755128901.000,1755128904.321,IR5211,1.366,2.081,1.428


In [57]:
qc_kin_position_df =qc_pipeline.qc_kin_position_data_dest.read_df(start=s)
qc_kin_position_df['dateTime'] = qc_kin_position_df.time
qc_kin_position_df.time = qc_kin_position_df.time.apply(lambda x:x.timestamp())
qc_kin_position_df['time']

0     1755043235.000
1     1755043355.000
2     1755043490.000
3     1755043610.000
4     1755043745.000
           ...      
778   1755129306.000
779   1755129426.000
780   1755129428.000
781   1755129546.000
782   1755129548.000
Name: time, Length: 783, dtype: float64

In [58]:
pingTimes = sorted(qcdata_df['pingTime'].unique())
pingTimes

[1755043235.0,
 1755043355.0,
 1755043490.0,
 1755043610.0,
 1755043745.0,
 1755043865.0,
 1755043985.0,
 1755044105.0,
 1755044225.0,
 1755044345.0,
 1755044465.0,
 1755044585.0,
 1755044705.0,
 1755044825.0,
 1755044945.0,
 1755045065.0,
 1755045185.0,
 1755045305.0,
 1755045425.0,
 1755045545.0,
 1755045665.0,
 1755045785.0,
 1755045905.0,
 1755046040.0,
 1755046160.0,
 1755046295.0,
 1755046415.0,
 1755046535.0,
 1755046655.0,
 1755046775.0,
 1755046895.0,
 1755047015.0,
 1755047136.0,
 1755047256.0,
 1755047391.0,
 1755047511.0,
 1755047631.0,
 1755047751.0,
 1755047886.0,
 1755048006.0,
 1755048126.0,
 1755048246.0,
 1755048366.0,
 1755048486.0,
 1755048606.0,
 1755048726.0,
 1755048846.0,
 1755048966.0,
 1755049086.0,
 1755049206.0,
 1755049341.0,
 1755049461.0,
 1755049581.0,
 1755049701.0,
 1755049821.0,
 1755049956.0,
 1755050076.0,
 1755050211.0,
 1755050331.0,
 1755050451.0,
 1755050571.0,
 1755050691.0,
 1755050811.0,
 1755050931.0,
 1755051051.0,
 1755051186.0,
 175505130

In [59]:
ping_pos_df = qc_kin_position_df[qc_kin_position_df['time'].isin(pingTimes)]
ping_pos_df=ping_pos_df[['time','east','north','up']].rename(columns={'east':'east0', 'north':'north0','up':'up0', 'time':'pingTime'}).set_index('pingTime')
ping_pos_df

,east0,north0,up0
pingTime,,,
1755043235.000,-2741333.454,-3953327.083,4173625.199
1755043355.000,-2741389.384,-3953286.188,4173625.260
1755043490.000,-2741441.905,-3953222.822,4173651.671
1755043610.000,-2741465.346,-3953156.142,4173699.169
1755043745.000,-2741449.503,-3953078.632,4173782.305
...,...,...,...
1755129036.000,-2740831.922,-3952566.911,4174667.339
1755129171.000,-2740815.209,-3952500.102,4174740.625
1755129306.000,-2740807.417,-3952437.343,4174806.201


In [ ]:
for col in ['east0','north0','up0']:
    mapper = ping_pos_df[col]
    new_pos_mapped = qcdata_df['pingTime'].map(mapper)
    qcdata_df_updated[col] = new_pos_mapped.combine_first(qcdata_df[col])
    qcdata_df_updated['isUpdated']

In [61]:
qcdata_df_updated

,pingTime,transponderID,head0,pitch0,roll0,head1,pitch1,roll1,east0,north0,...,east_std1,north_std1,up_std1,returnTime,tt,dbv,xc,snr,tat,isUpdated
0,1755043235.000,IR5209,231.515,-1.093,2.938,234.359,1.085,0.824,-2741333.454,-3953327.083,...,7.791,9.097,23.000,1755043237.646,2.446,-26,88,-3.000,0.200,0
1,1755043355.000,IR5209,251.405,-1.823,0.761,243.732,-0.624,-1.449,-2741389.384,-3953286.188,...,7.773,9.126,23.000,1755043357.647,2.447,-25,86,30.000,0.200,0
2,1755043490.000,IR5209,270.209,1.621,1.289,260.865,2.179,-1.474,-2741441.905,-3953222.822,...,8.963,10.320,23.000,1755043492.620,2.420,-24,87,15.000,0.200,0
3,1755043610.000,IR5209,290.659,-1.561,0.711,297.256,0.943,2.361,-2741465.346,-3953156.142,...,8.953,10.266,23.000,1755043612.567,2.367,-25,91,31.000,0.200,0
4,1755043745.000,IR5209,330.694,-0.440,1.634,322.297,1.662,-2.183,-2741449.503,-3953078.632,...,8.887,10.225,23.000,1755043747.472,2.272,-21,79,22.000,0.200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,1755128526.000,IR5211,328.422,4.358,-0.358,332.253,-2.097,2.204,-2740983.490,-3952785.014,...,8.181,10.006,23.000,1755128529.080,2.640,-30,69,9.000,0.440,0
1338,1755128661.000,IR5211,335.374,6.832,-2.135,329.545,1.554,-0.995,-2740942.072,-3952727.544,...,7.785,9.009,23.000,1755128664.161,2.721,-34,65,10.000,0.440,0
1339,1755128781.000,IR5211,328.410,-3.194,5.020,339.906,-1.567,6.082,-2740902.994,-3952672.578,...,7.791,9.027,23.000,1755128784.244,2.804,-35,72,10.000,0.440,0
1340,1755128901.000,IR5211,331.807,-0.203,5.416,330.001,1.435,-0.040,-2740871.929,-3952625.295,...,7.809,9.062,20.700,1755128904.321,2.881,-36,70,7.000,0.440,0


In [67]:
new_pos_mapped

0      4173625.199
1      4173625.260
2      4173651.671
3      4173699.169
4      4173782.305
           ...    
1337   4174363.987
1338   4174444.394
1339   4174521.217
1340   4174587.811
1341   4174667.339
Name: pingTime, Length: 1342, dtype: float64

In [62]:
qcdata_df['east0']-qcdata_df_updated['east0']

0       0.227
1       0.636
2       0.588
3       0.133
4      -0.023
        ...  
1337   -0.598
1338   -0.345
1339   -0.406
1340   -0.314
1341    0.012
Name: east0, Length: 1342, dtype: float64

In [18]:
kinTimes = sorted(qc_kin_position_df['time'].unique())
kinTimes

[1755043235.0,
 1755043355.0,
 1755043490.0,
 1755043610.0,
 1755043745.0,
 1755043865.0,
 1755043985.0,
 1755044105.0,
 1755044225.0,
 1755044345.0,
 1755044465.0,
 1755044585.0,
 1755044705.0,
 1755044825.0,
 1755044945.0,
 1755045065.0,
 1755045185.0,
 1755045305.0,
 1755045425.0,
 1755045545.0,
 1755045665.0,
 1755045785.0,
 1755045788.0,
 1755045905.0,
 1755045908.0,
 1755046040.0,
 1755046160.0,
 1755046295.0,
 1755046415.0,
 1755046535.0,
 1755046655.0,
 1755046775.0,
 1755046895.0,
 1755047015.0,
 1755047136.0,
 1755047139.0,
 1755047256.0,
 1755047259.0,
 1755047391.0,
 1755047511.0,
 1755047631.0,
 1755047751.0,
 1755047886.0,
 1755048006.0,
 1755048126.0,
 1755048246.0,
 1755048366.0,
 1755048486.0,
 1755048606.0,
 1755048726.0,
 1755048846.0,
 1755048966.0,
 1755049086.0,
 1755049206.0,
 1755049341.0,
 1755049461.0,
 1755049581.0,
 1755049701.0,
 1755049821.0,
 1755049956.0,
 1755050076.0,
 1755050211.0,
 1755050331.0,
 1755050451.0,
 1755050571.0,
 1755050691.0,
 175505081

In [21]:
for ping in pingTimes:
    if ping not in kinTimes:
        print(ping)